### Initialize notebook variables

Configures everything that's needed for deployment. **Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.SIMPLE_APIM
tags        = ['general']

# 2) Service-defined parameters (please do not change these)
rg_name       = utils.get_infra_rg_name(deployment, index)
sample_folder = "general"
nb_helper     = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM])

# 3) Define the APIs and their operations and policies

# API 1
api1_get = GET_APIOperation('This is a GET for API 1')
api1_post = POST_APIOperation('This is a POST for API 1')
api1 = API('API1', 'API 1', '/api1', 'This is API 1', operations = [api1_get, api1_post], tags = tags)

# API 2
api2_post = POST_APIOperation('This is a POST for API 2')
api2 = API('API2', 'API 2', '/api2', 'This is API 2', operations = [api2_post], tags = tags)

# Request Headers
rh_policy_xml = utils.read_policy_xml(REQUEST_HEADERS_XML_POLICY_PATH)
rh_get = GET_APIOperation('Gets the request headers for the current request and returns them. Great for troubleshooting.', rh_policy_xml)
rh = API('requestheaders', 'Request Headers', '/request-headers', 'API for request headers', operations = [rh_get], tags = tags)

# APIs Array
apis: List[API] = [api1, api2, rh]

utils.print_ok('Notebook initialized')

### Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis'   : { 'value': [api.to_dict() for api in apis] }
}

# 2) Deploy the bicep template
output = nb_helper.deploy_bicep(bicep_parameters)

if output.json_data:
    apim_name           = output.get('apimServiceName', 'APIM Service Name')
    apim_gateway_url    = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')

### Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
import utils
from apimrequests import ApimRequests
from apimtesting import ApimTesting

tests = ApimTesting("General Sample Tests", sample_folder, deployment)

# Check if the infrastructure architecture deployment uses Azure Front Door.
utils.print_message('Checking if the infrastructure architecture deployment uses Azure Front Door.', blank_above = True)
afd_endpoint_url = utils.get_frontdoor_url(deployment, rg_name)

if afd_endpoint_url:
    reqsAfd = ApimRequests(afd_endpoint_url)
    output = reqsAfd.singleGet('/request-headers', msg = 'Calling Request Headers API via via Azure Front Door. Expect 200.')
    tests.verify('Host:' in output, True)
else:
    # Issue a direct request to API Management
    reqsApim = ApimRequests(apim_gateway_url)
    output = reqsApim.singleGet('/request-headers', msg = 'Calling Request Headers API via API Management Gateway URL. Response codes 200 and 403 are both valid depending on the infrastructure used.')

tests.print_summary()

utils.print_ok('All done!')